In [3]:
import requests
import pandas as pd
from datetime import timedelta
from datetime import datetime

from airflow import DAG
from airflow.operators.python import PythonOperator


In [4]:
TOP_1M_DOMAINS = 'http://s3.amazonaws.com/alexa-static/top-1m.csv.zip'
TOP_1M_DOMAINS_FILE = 'top-1m.csv'


def get_data():
    # Здесь пока оставили запись в файл, как передавать переменую между тасками будет в третьем уроке
    top_doms = pd.read_csv(TOP_1M_DOMAINS)
    top_data = top_doms.to_csv(index=False)

    with open(TOP_1M_DOMAINS_FILE, 'w') as f:
        f.write(top_data)


In [5]:
get_data()

In [6]:
def get_domen():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    top_domen_10 = top_data_df.domain.str.split('.', expand=True)[1].value_counts().head(10)
    with open('top_domen_10.csv', 'w') as f:
        f.write(top_domen_10.to_csv( header=False))
    

In [7]:
def print_data(ds):
    top_domen_10 = pd.read_csv('top_domen_10.csv', header=None)
    longest_domain = pd.read_csv('longest_domain.csv')
    place_airflow = pd.read_csv('place_airflow.csv')
        
    date = ds

    print(f'Top domains for date {date}')
    print(top_domen_10[0])
    
    print(f'Domain with longest name {date}')
    print(longest_domain.domain[0])
    
    print(f'Airflow place {date}')
    if place_airflow.shape[0] == 0:
        print('no data about - airflow.com')
    else:
        print('place_airflow.place[0]')

In [11]:
def get_longest_domain():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    longest_domain = \
    (top_data_df
     .assign(
     domain_len = lambda x: x.domain.apply(len)
     )
     .sort_values(['domain_len', 'domain'], ascending = [False, False])
     .head(1)
     .loc[:,['domain','domain_len' ]]
    )
    with open('longest_domain.csv', 'w') as f:
        f.write(longest_domain.to_csv( header=True, index = False))

In [16]:
get_longest_domain()

In [36]:
def place_airflow():
    top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
    longest_domain = \
    (top_data_df
     .assign(
     domain_len = lambda x: x.domain.apply(len)
     )
     .sort_values(['domain_len', 'domain'], ascending = [False, False])
     .assign(
         place = lambda x: range(1,len(x)+1)
     )
     .query("domain == 'airflow.com'")
    )
    with open('place_airflow.csv', 'w') as f:
        f.write(longest_domain.to_csv( header=True, index = False))

In [37]:
place_airflow()

In [40]:
place_airflow = pd.read_csv('place_airflow.csv')

In [43]:
print(f'Airflow place {date}')
if place_airflow.shape[0] == 0:
    print('no data about - airflow.com')
else:
    print('place_airflow.place[0]')
    

0

In [47]:
place_airflow.place[0]

Series([], Name: place, dtype: object)

In [28]:
top_data_df = pd.read_csv(TOP_1M_DOMAINS_FILE, names=['rank', 'domain'])
# longest_domain = \
#     (top_data_df
#      .assign(
#      domain_len = lambda x: x.domain.apply(len)
#      )
#      .sort_values(['domain_len', 'domain'], ascending = [False, False])
#      .head(1)
#      .loc[:,['domain','domain_len' ]]
#     )

In [32]:
(top_data_df
 .assign(
 domain_len = lambda x: x.domain.apply(len)
 )
 .sort_values(['domain_len', 'domain'], ascending = [False, False])
 .assign(
     place = lambda x: range(1,len(x)+1)
 )
 .query("domain == 'airflow.com'")
)

,rank,domain,domain_len,place
